<a href="https://colab.research.google.com/github/greysat/SentimentPulse/blob/main/Scraping_data_from_GooglePlayStore_(Solitaire_Showdown_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install -q google_play_scraper #scraping reviews
!pip install -q transformers #sentiment
!pip install -q plotly-express #data visualization

In [22]:
!pip install pyyaml

In [3]:
import pandas as pd
import numpy as np
from google_play_scraper import app, reviews_all
import plotly.express as px

In [25]:
solitaire_showdown_project = reviews_all( 'air.com.qublix.showdowntwo', sleep_milliseconds=0, lang='en', country='US')

In [26]:
solitaire_showdown_project

[{'reviewId': '1c1a1b64-7919-45d1-bb0e-207fd179221d',
  'userName': 'Andrew ẞarkys',
  'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjVk1nJeTsN-FJFQI4UdLLTQfIw9CIs4aGj6RYubonBtrA',
  'content': 'Smart play, the game is interesting!',
  'score': 5,
  'thumbsUpCount': 0,
  'reviewCreatedVersion': '1.3.62',
  'at': datetime.datetime(2023, 6, 19, 23, 4, 16),
  'replyContent': None,
  'repliedAt': None,
  'appVersion': '1.3.62'},
 {'reviewId': '558f5e8c-2cf2-42e7-a4a5-a1b50f0c42f7',
  'userName': 'Sherry Tingle',
  'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjWIsWDAjtOzkb4i7JUQWHCkjhtRQ4I9tTsGL8PT2iunppA',
  'content': "Thanks for my refund. This is one of my number 1 games. I occasionally buy bonus tickets for this game. But now I'm afraid this might happen again. Can you reassure me that this won't happen again? I need to have my bonus tickets when I catch a player thinking they're slicker than boiled Okra. I wanna buy some more tickets...but I'm scared 😱",
 

In [28]:
df = pd.json_normalize(solitaire_showdown_project)

In [29]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,1c1a1b64-7919-45d1-bb0e-207fd179221d,Andrew ẞarkys,https://play-lh.googleusercontent.com/a-/ALV-U...,"Smart play, the game is interesting!",5,0,1.3.62,2023-06-19 23:04:16,None,NaT,1.3.62
1,558f5e8c-2cf2-42e7-a4a5-a1b50f0c42f7,Sherry Tingle,https://play-lh.googleusercontent.com/a-/ALV-U...,Thanks for my refund. This is one of my number...,5,2,1.3.0,2023-05-18 18:40:10,"Hello,\n\nWe are very sorry for the issue, and...",2023-05-23 13:50:02,1.3.0
2,bac06397-7448-4608-a5b6-cefaf7b41271,charles archer,https://play-lh.googleusercontent.com/a-/ALV-U...,I sense a little bit of a problem arising in t...,3,2,1.3.62,2023-05-18 01:59:01,None,NaT,1.3.62
3,e25d7a66-a64c-4f94-b058-bfae510617fb,renea day,https://play-lh.googleusercontent.com/a-/ALV-U...,Game fun n no new levels same levels done pass...,2,5,1.3.63,2023-04-25 23:29:47,Our development team will definitely consider ...,2023-05-05 15:43:49,1.3.63
4,001d97e5-7ecc-4026-9bbb-e73836830a00,Lisabeth Dixon,https://play-lh.googleusercontent.com/a/ACg8oc...,LisabethDixon Save,5,0,None,2023-02-01 17:39:39,None,NaT,None


In [30]:
df['score'].mean()

4.123076923076923

In [31]:
df['reviewCreatedVersion'].value_counts()

0.0.9     55
1.3.3     22
1.3.2      8
1.3.41     5
1.2.0      5
1.3.0      4
1.3.40     4
1.3.62     2
1.3.63     1
Name: reviewCreatedVersion, dtype: int64

In [32]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

In [33]:
df['content'] = df['content'].astype('str')

In [34]:
df['result'] = df['content'].apply(lambda x: sentiment_analysis(x))

In [35]:
data = df.to_dict(orient='records')

In [36]:
df['sentiment']=df['result'].apply(lambda x:(x[0]['label']))
df['score']=df['result'].apply(lambda x:(x[0]['score']))

In [37]:
df['sentiment'].value_counts()

POSITIVE    92
NEGATIVE    38
Name: sentiment, dtype: int64

In [38]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,result,sentiment
0,1c1a1b64-7919-45d1-bb0e-207fd179221d,Andrew ẞarkys,https://play-lh.googleusercontent.com/a-/ALV-U...,"Smart play, the game is interesting!",0.998779,0,1.3.62,2023-06-19 23:04:16,None,NaT,1.3.62,"[{'label': 'POSITIVE', 'score': 0.998778998851...",POSITIVE
1,558f5e8c-2cf2-42e7-a4a5-a1b50f0c42f7,Sherry Tingle,https://play-lh.googleusercontent.com/a-/ALV-U...,Thanks for my refund. This is one of my number...,0.999247,2,1.3.0,2023-05-18 18:40:10,"Hello,\n\nWe are very sorry for the issue, and...",2023-05-23 13:50:02,1.3.0,"[{'label': 'NEGATIVE', 'score': 0.999246954917...",NEGATIVE
2,bac06397-7448-4608-a5b6-cefaf7b41271,charles archer,https://play-lh.googleusercontent.com/a-/ALV-U...,I sense a little bit of a problem arising in t...,0.982992,2,1.3.62,2023-05-18 01:59:01,None,NaT,1.3.62,"[{'label': 'NEGATIVE', 'score': 0.982992231845...",NEGATIVE
3,e25d7a66-a64c-4f94-b058-bfae510617fb,renea day,https://play-lh.googleusercontent.com/a-/ALV-U...,Game fun n no new levels same levels done pass...,0.998712,5,1.3.63,2023-04-25 23:29:47,Our development team will definitely consider ...,2023-05-05 15:43:49,1.3.63,"[{'label': 'POSITIVE', 'score': 0.998712182044...",POSITIVE
4,001d97e5-7ecc-4026-9bbb-e73836830a00,Lisabeth Dixon,https://play-lh.googleusercontent.com/a/ACg8oc...,LisabethDixon Save,0.995623,0,None,2023-02-01 17:39:39,None,NaT,None,"[{'label': 'POSITIVE', 'score': 0.995623171329...",POSITIVE


In [39]:
fig = px.histogram(df, x='sentiment', color='sentiment', text_auto=True)
fig.show()

In [40]:
data

[{'reviewId': '1c1a1b64-7919-45d1-bb0e-207fd179221d',
  'userName': 'Andrew ẞarkys',
  'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjVk1nJeTsN-FJFQI4UdLLTQfIw9CIs4aGj6RYubonBtrA',
  'content': 'Smart play, the game is interesting!',
  'score': 5,
  'thumbsUpCount': 0,
  'reviewCreatedVersion': '1.3.62',
  'at': Timestamp('2023-06-19 23:04:16'),
  'replyContent': None,
  'repliedAt': NaT,
  'appVersion': '1.3.62',
  'result': [{'label': 'POSITIVE', 'score': 0.9987789988517761}]},
 {'reviewId': '558f5e8c-2cf2-42e7-a4a5-a1b50f0c42f7',
  'userName': 'Sherry Tingle',
  'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjWIsWDAjtOzkb4i7JUQWHCkjhtRQ4I9tTsGL8PT2iunppA',
  'content': "Thanks for my refund. This is one of my number 1 games. I occasionally buy bonus tickets for this game. But now I'm afraid this might happen again. Can you reassure me that this won't happen again? I need to have my bonus tickets when I catch a player thinking they're slicker than boiled Ok